In [1]:
#!pip install lightgbm

In [2]:
import pandas as pd 
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import time

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier 

import matplotlib.pyplot as plt
import seaborn as sn

In [3]:
from lightgbm import LGBMClassifier

In [4]:
train=pd.read_csv('G:\\My Drive\\2022 Course DPHI\\Microbes_train_dataset.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'G:\\My Drive\\2022 Course DPHI\\Microbes_train_dataset.csv'

**ANALYSIS**

In [ ]:
train.head(2)

In [ ]:
train=train.drop('Serial No', axis=1)
train.shape

In [ ]:
train.info()

In [ ]:
#train.isnull().sum()

In [ ]:
train.describe()

In [ ]:
train.describe(include=object)

In [ ]:
#train['microorganisms'].nunique()

In [ ]:
#train['microorganisms']

**DIVIDE DATASET INTO TWO TO VIEW FEATURE RELATIONSHIP WITH TARGET (OPTIONAL)**

In [ ]:
l=len(train.columns)

train1=train.iloc[:, :(l/2).__ceil__()]
train2=train.iloc[:, (l/2).__ceil__():]

In [ ]:
train1[['microorganisms']]=train[['microorganisms']]
train1.shape

In [ ]:
train1.corr()

In [ ]:
train2.corr()

**ANALYSIS 2**

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mm=MinMaxScaler()
sc=StandardScaler()
rs = RobustScaler()
from sklearn.preprocessing import LabelEncoder

In [ ]:
new_data=train

In [ ]:
#DEFINE THE DIFFERENT SET OF COLUMNS WE WILL USE
cols= train.columns
number_cols = train.select_dtypes(include=np.number).columns
cat_cols= list(set(cols) - set(number_cols)) 

In [ ]:
new_data['microorganisms'].value_counts()

In [ ]:
plt.figure(figsize=(5, 3))
new_data['microorganisms'].value_counts().plot.bar()
plt.show()

In [ ]:
#USE HEATMAP TO SEE RELATIONSHIP (correlation) BETWEEN TARGET AND PREDICTION VARIABLES
plt.figure(figsize=(15, 6))
sn.heatmap(train[number_cols].corr(), annot=True, fmt='.3f', yticklabels=number_cols, xticklabels=number_cols)
plt.title('Covariance for correlation coefficients')
plt.tight_layout()
plt.show()

**SELECT FEATURES THAT ARE NOT HILGHLY CORELATED AND DRAW THE CORELATION MATRIX AGAIN**

In [ ]:
sel_col=['Eccentricity',  'FilledArea','Extent', 'Orientation', 'EulerNumber','BoundingBox2', 'BoundingBox4', 'ConvexHull3', 
 'MajorAxisLength', 'MinorAxisLength', 'Perimeter', 'ConvexArea', 'Area', 'raddi','microorganisms']
train_reduced=train[sel_col]
len(sel_col)

In [ ]:
plt.figure(figsize=(10, 5))
sn.heatmap(train_reduced.corr(), annot=True, fmt='.2f', yticklabels=sel_col, xticklabels=sel_col)
plt.title('Covariance for correlation coefficients')
plt.tight_layout()
plt.show()

In [ ]:
#DISTRIBUTION OF THE VARIABLES
p = sn.catplot(data = new_data,height=4, aspect=16/4)
plt.title('Range (min-max) of data distribution within features')
plt.show()

In [ ]:
#DISTRIBUTION OF THE VARIABLES WITH REDUCED COLUMNS
p = sn.catplot(data = train_reduced,height=4, aspect=16/4)
plt.title('Range (min-max) of data distribution within reduced features')
plt.show()

You can see that most of the features are highly cluster alon the distribution line, but a few are sparse. Notice that the max value of all the features are equal, so we dont need a min-max scaler

In [ ]:
#PLOT OF THE DISTRIBUTION OF DATA IN FEATURES                                    
new_data.hist(alpha=0.5, figsize=(9, 5), bins=10, facecolor='grey')
plt.tight_layout()
plt.show()

In [ ]:
# PLOT OF THE DISTRIBUTION OF DATA IN SELECTED FEATURES                                    
train_reduced.hist(alpha=0.5, figsize=(8, 4), bins=10, facecolor='grey')
plt.tight_layout()
plt.show()

Some of the features have guasian distribution (not very skewed), but there are outliers, while some of the features are skewed, so we will use data standardization for scaling this dataset. But first, lets see these distributions with a box plot

In [ ]:
# BOXPLOT OF THE ACTUAL DATASET  UNDERSTAND THE ABOVE
plt.figure(figsize=(18, 4))
new_data.boxplot()
plt.show()

In [ ]:
#BOXPLOT OF THE DATASET WITH REDUCED COLUMNS
plt.figure(figsize=(18, 4))
train_reduced.boxplot()
plt.show()

See a clear distribution of the data features. Most of the upper values are outliers. and only a few of the features are on the same scale

**DATASET PREPARATION: REDUCED FEATURES DATASET**

In [ ]:
#DEFINE X AND Y FOR THE REDUCED FEATURES
y_reduced=train_reduced['microorganisms']
y_reduced2 = LabelEncoder().fit_transform(y_reduced.astype('str'))
x_reduced=train_reduced.drop('microorganisms',axis =1)

In [ ]:
#SCALE THE DATASET ON REDUCED FEATURES (STANDARD SCALER)
s = sc.fit(x_reduced.values)
Xrs= s.transform(x_reduced.values)

In [ ]:
#SCALE THE DATASET ON REDUCED FEATURES (ROBUST SCALER)
r=rs.fit(x_reduced.values)
Xrr=r.transform(x_reduced.values)

**Make boxplots to get a clearer view of the scaled distribution**

In [ ]:
plt.figure(figsize=(18, 4))
Xrs.boxplot()
plt.show()

In [ ]:
plt.figure(figsize=(18, 4))
Xrr.boxplot()
plt.show()

**Obviously, he robust scaler did a better job, but the result will tell. Now lets do the same for the complete features**

**DATASET PREPARATION: COMPLETE FEATURES DATASET**

In [ ]:
# DEFINE X AND Y FOR THE COMPLETE FEATURES
y=train['microorganisms']
y1 = LabelEncoder().fit_transform(y.astype('str'))
x=train.drop('microorganisms', axis =1)

In [ ]:
y=y.astype("category")
#y

In [ ]:
#SCALE THE DATASET ON (ROBUST SCALER)
r1=rs.fit(x.values)
Xr=r1.transform(x.values)

In [ ]:
#SCALE THE DATASET ON (STANDARD SCALER)
s1=sc.fit(x.values)
Xs=s1.transform(x.values)

In [ ]:
#SCALE THE DATASET ON (STANDARD SCALER)
m1=mm.fit(x.values)
Xm=m1.transform(x.values)

In [ ]:
plt.figure(figsize=(18, 4))
Xr.boxplot()
plt.show()

In [ ]:
plt.figure(figsize=(18, 4))
Xs.boxplot()
plt.show()

In [ ]:
#mimmax scaled
m2=mm.fit(x)
Xm2=m2.transform(x)
mx=pd.DataFrame(data=Xm2)
plt.figure(figsize=(18, 4))
mx.boxplot()
plt.show()

Again the robust scaler performed better, as all the features are now a on a single mean

**SPLITTING DATASET**

In [ ]:
# Splitting the dataset into Training set and Test set
from sklearn.model_selection import train_test_split
#Splitting full dataset
Xf_train,Xf_test,yf_train,yf_test=train_test_split(x,y,test_size=0.3,random_state=0)#full unscaled

Xr_train,Xr_test,yr_train,yr_test=train_test_split(Xr,y,test_size=0.3,random_state=0)
Xs_train,Xs_test,ys_train,ys_test=train_test_split(Xs,y,test_size=0.3,random_state=0)
Xm_train,Xm_test,ym_train,ym_test=train_test_split(Xm,y,test_size=0.3,random_state=0)

In [ ]:

#Splitting reduced column dataset
Xrs_train,Xrs_test,yrs_train,yrs_test=train_test_split(Xrs,y_reduced2,test_size=0.3,random_state=0)
Xrr_train,Xrr_test,yrr_train,yrr_test=train_test_split(Xrr,y_reduced2,test_size=0.3,random_state=0)

**CREATING AND RUNNING THE MODELS**

In [ ]:
#START WITH THE BASELINE MODELS (TESTING 8 MODELS)
classifiers2=[GradientBoostingClassifier(),GaussianNB(), DecisionTreeClassifier(),MLPClassifier(), 
KNeighborsClassifier(), RandomForestClassifier(), CatBoostClassifier(logging_level='Silent'), LogisticRegression(max_iter=1000)]

In [ ]:
#START WITH THE BASELINE MODELS (TESTING 1 MODEL)
#classifiers2=[XGBClassifier()]
classifiers=[LGBMClassifier()]

**DEFINE FUNCTION TO RUN MODELS**

In [ ]:
def squared_error(actual, pred):
    return (pred - actual) ** 2

def modeld(classifiers, X_train, y_train, X_test, y_test):  
    model_lisdt=[]
    r_list=[]  
    for classers in classifiers:
        
        t1=time.time()
        classify=classers
        classify = classify.fit(X_train, y_train)
        r_list.append(classify)
        time_taken = time.time() - t1
        
        predicted=classify.predict(X_test)        
        st=str(classers)
              
        print(' \n ############## {} ################'.format(st))
        print(" Duration =:\t {}".format(time_taken))
        print(" Accuracy =: \t {:0.2f} %".format(accuracy_score(y_test, predicted)*100))      
    return(r_list)

**RUN ALL MODELS ON COMPLETE COLUMNS (USING ALL THE FEATURES)**

*TRYING MINMAX SCALER*

In [ ]:
standarsised_full=modeld(classifiers, Xs_train,ys_train,Xs_test,ys_test)
unscaled_full=modeld(classifiers, Xf_train,yf_train, Xf_test,yf_test)
robustscaled_full= modeld(classifiers, Xr_train,yr_train,Xr_test,yr_test)
min_max_full=modeld(classifiers, Xm_train, ym_train, Xm_test, ym_test)


In [ ]:
modele=min_max_full[0]
prede=modele.predict(Xm_test)
print(" Accuracy =: \t {:0.4f} %".format(accuracy_score(ym_test, prede)*100))
 # unscaled_full=modeld(classifiers, Xf_train,yf_train, Xf_test,yf_test)

*USING ALL SCALERS AND MODELS*

In [ ]:
unscaled_full=modeld(classifiers, Xf_train,yf_train, Xf_test,yf_test)

In [ ]:
robustscaled_full= modeld(classifiers, Xr_train,yr_train,Xr_test,yr_test)

In [ ]:
standarsised_full=modeld(classifiers, Xs_train,ys_train,Xs_test,ys_test)

In [ ]:
min_max_full=modeld(classifiers, Xm_train, ym_train, Xm_test, ym_test)

**RUN ALL MODELS ON DATASET USED REDUCED FEATURES**

In [ ]:
robustscaled_reduced= modeld(classifiers, Xrr_train,yrr_train,Xrr_test,yrr_test)

In [ ]:
standarsised_reduced=modeld(classifiers, Xrs_train,yrs_train,Xrs_test,yrs_test)

# OBSERVATIONS
After the modelling, it was noticed that the reduced features data did not perform well at all.
Howerever, using the full features bothe scaled and unscaled, showed a very efficient performance, notwithstanding, both robust and standard scler seem to perform better than non scaled.

Ultimately, the best performance was from GradientBoostingClassifier(), on standard-scaler scaled data.
Other better performce include:

CatBoostClassifier(), 
DecisionTreeClassifier(),
RandomForestClassifier()

**SELECT BEST MODEL TO PREDICT EVALUATION DATASET**

In [ ]:
# STANDARDISED FULL FEATURE RESULT

model=standarsised_full[6]
pred=model.predict(Xs_test)
print(" Accuracy =: \t {:0.2f} %".format(accuracy_score(ys_test, pred)*100))  

'''
Accuracy =: 	 99.98 % #GB
Accuracy =: 	 99.97 % #DT
Accuracy =: 	 99.84 % #RF
Accuracy =: 	 99.97 % #CB
'''

In [ ]:
# ROBUST FULL FEATURE RESULT

model2=robustscaled_full[5]
pred2=model2.predict(Xr_test)
print(" Accuracy =: \t {:0.2f} %".format(accuracy_score(yr_test, pred2)*100))#Xr_train,yr_train,Xr_test,yr_test

'''
Accuracy =: 	 99.97 % #GB
Accuracy =: 	 99.95 % #DT
Accuracy =: 	 99.92 % #RF
Accuracy =: 	 99.97 % #CB
''' 

In [ ]:
l=list(set(cols) - set(['microorganisms'])) 
feat_importances = pd.Series(model.feature_importances_, index=l)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

**EVALUATION*

**TESTING THE EVALUATION DATASET**

In [ ]:
test_d=pd.read_csv('G:\\My Drive\\2022 Course DPHI\\Microbes_test_dataset.csv')

In [ ]:
#test.isnull().sum()

In [ ]:
test_d.shape

In [ ]:
#STANDARD
s5=sc.fit(test_d.values)
Xtt =s5.transform(test_d.values)

In [ ]:
#ROBUST
r5=rs.fit(test_d.values)
Xr5 =r5.transform(test_d.values)

In [ ]:
#MIN-MAX
m5=mm.fit(test_d.values)
Xm5 =m5.transform(test_d.values)
#modele=min_max_full[0]

In [ ]:
predte=modele.predict(Xm5)
predte

In [ ]:
datafe=pd.DataFrame(data=predte, columns=['microorganisms'])
datafe

In [ ]:
#fde=pd.DataFrame(data=datafe, columns=['microorganisms'])

In [ ]:
                                  #   CONVERT FINAL DATASFRAME TO CSV
filename='glsam_datathonMICROBEee'
datafe.to_csv('G:\\My Drive\\2022 Course DPHI\\{}'.format(filename + '.csv'), sep=',', header=True, index=False)